In [1]:
import librosa as lr
import pandas as pd
import numpy as np
import os
import scipy
from tqdm.notebook import tqdm

Змінні

In [2]:
h_audio_path = "C:\\audio_classificator\\dataset\\chunks\\human"
b_audio_path = "C:\\audio_classificator\\dataset\\chunks\\bot"

h_filenames = os.listdir(h_audio_path)
b_filenames = os.listdir(b_audio_path)

cols = ['audio_name'] + [f'mfcc_mean_{i}' for i in range(20)] + \
       [f'mfcc_std_{i}' for i in range(20)] + \
       ['mean_sc', 'std_sc', 'skew_sc', 'mean_sr', 'std_sr', 'mean_sb', 'std_sb', 'is_bot']
feature_list = []


In [3]:
def get_feature(directory, f_name):
    try:
        y, sr = lr.load(f"{directory}\\{f_name}")
        
        features = [] 
        features.append(f_name)
        features.extend([np.mean(e) for e in lr.feature.mfcc(y=y, sr=sr, n_mfcc=20)])  
        features.extend([np.std(e) for e in lr.feature.mfcc(y=y, sr=sr, n_mfcc=20)])
        
        features.append(np.mean(lr.feature.spectral_centroid(y=y, sr=sr).T, axis = 0)[0])     
        features.append(np.std(lr.feature.spectral_centroid(y=y,sr=sr).T, axis = 0)[0])
        features.append(scipy.stats.skew(lr.feature.spectral_centroid(y=y,sr=sr).T, axis = 0)[0])
        
        features.append(np.mean(lr.feature.spectral_rolloff(y=y, sr=sr).T, axis = 0)[0])     
        features.append(np.std(lr.feature.spectral_rolloff(y=y, sr=sr).T, axis = 0)[0])    
    
        features.append(np.mean(lr.feature.spectral_bandwidth(y=y, sr=sr).T, axis = 0)[0])     
        features.append(np.std(lr.feature.spectral_bandwidth(y=y, sr=sr).T, axis = 0)[0])
    
        features.append(0 if directory.split('\\')[-1] == 'human' else 1)
        return features
    except Exception:
        return None

In [ ]:
for directory, f_name in [(h_audio_path, tqdm(h_filenames)), (b_audio_path, tqdm(b_filenames))]:
    for f in f_name:
        res = get_feature(directory, f)
        if res is not None:
            feature_list.append(res)

  0%|          | 0/3930 [00:00<?, ?it/s]

  0%|          | 0/4229 [00:00<?, ?it/s]

In [ ]:
features_csv = pd.DataFrame(feature_list, columns=cols)

In [ ]:
features_csv.to_csv('C:\\audio_classificator\\csv\\features.csv', index=False)